### Función para descomprimir los datos del socket

In [15]:
def descomprimirDatosWeb(compressed_str):
    compressed = [ord(c) for c in compressed_str]
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    result = []
    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError(f"Bad compressed k: {k}")
        result.append(entry)

        # Add w+entry[0] to the dictionary
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry

    return "".join(result)


### Funcion para sacar el pais con lat y lon

In [16]:
import reverse_geocoder as rg

def obtenerPais(lat, lon):
    try:
        pais = rg.search((lat, lon))
        return pais[0]['name']
    except:
        return 'Desconocido'


### Funcion para limpiar

In [17]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo


def limpiarDatos(d):
    
    timeNs = d.get("time")

    try:
        # Convertir nanosegundos a segundos
        timeSeg = int(timeNs) / 1e9
        dt = datetime.fromtimestamp(timeSeg, tz=timezone.utc)
        horaEsp = dt.astimezone(ZoneInfo("Europe/Madrid"))
        
        fecha = dt.strftime("%Y-%m-%d")
        hora = horaEsp.strftime("%H:%M:%S")
        
    except Exception as e:
        print("⚠️ Error al convertir timestamp:", timeNs, e)
        fecha = "1970-01-01"
        hora = "00:00:00"
    
    pais = obtenerPais(d.get("lat"),d.get("lon"))


    datosLimpios = {
        "time": int(timeNs),
        "lat": d.get("lat"),
        "lon": d.get("lon"),
        "alt": d.get("alt"),
        "pol": d.get("pol"),
        "mds": d.get("mds"),
        "mcg": d.get("mcg"),
        "status": d.get("status"),
        "region": d.get("region"),
        "delay": d.get("delay"),
        "lonc": d.get("lonc"),
        "latc": d.get("latc"),
        "fecha": fecha,
        "hora": hora,
        "pais": pais,
        "estaciones": d.get("sig", [])
    }

    return datosLimpios


### Función para insertar los rayos

In [18]:
import mysql.connector

def insertarRayo(data):
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="rayos"
        )
        
        cursor = conn.cursor()
        
        cursor.execute("""
            INSERT INTO rayos (time, lat, lon, alt, pol, mds, mcg, status, region, delay, lonc, latc, fecha, hora, pais)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            data.get("time"),
            data.get("lat"),
            data.get("lon"),
            data.get("alt"),
            data.get("pol"),
            data.get("mds"),
            data.get("mcg"),
            data.get("status"),
            data.get("region"),
            data.get("delay"),
            data.get("lonc"),
            data.get("latc"),
            data.get("fecha"),
            data.get("hora"),
            data.get("pais"),
        ))
        
        idRayo = cursor.lastrowid
        
        # Insertar hasta 5 estaciones asociadas a este rayo
        for cont, estacion in enumerate(data.get("estaciones", [])):
            if cont >= 5:
                break  # solo guardar 5 estaciones como máximo
            cursor.execute("""
                INSERT INTO estaciones (idRayo, sta, tiempoRelat, lat, lon, alt, status)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                idRayo,
                estacion.get("sta"),
                estacion.get("time"),
                estacion.get("lat"),
                estacion.get("lon"),
                estacion.get("alt"),
                estacion.get("status")
            ))
        
        conn.commit()
        cursor.close()
        conn.close()
        
    except Exception as e:
        print("❌ Error al guardar en MySQL:", e)


# Lectura del socket y guardado a MYSQL XAMPP


In [19]:
import json
import aiohttp
import asyncio

async def escuchar():
    url = "wss://ws2.blitzortung.org/"
    headers = {
        "Origin": "https://www.blitzortung.org",
        "User-Agent": "Mozilla/5.0",
    }

    session = aiohttp.ClientSession()
    contRayos = 0

    try:
        async with session.ws_connect(url, headers=headers) as ws:
            await ws.send_str('{"a":111}')  # handshake inicial

            async for msj in ws:
                if msj.type == aiohttp.WSMsgType.TEXT:
                    
                    try:
                        raw = msj.data
                        # Funcion para descomprimir los datos
                        texto = descomprimirDatosWeb(raw)
                        data = json.loads(texto)
                        # Funcion para limpiar todos los datos y añadir otros
                        rayoLimp = limpiarDatos(data)
                        
                        if rayoLimp is not None:
                            # Funcion para inserte el rayo en MYSQL
                            insertarRayo(rayoLimp)
                            contRayos += 1
                            print(f"✅ Rayo {contRayos} guardado")
                        
                        # contador
                        if contRayos >= 10:
                            print("✅ Se han procesado 10 rayos.")
                            break
                        
                    except Exception as e:
                        print("❌ Error al procesar mensaje:", e)
                        
                elif msj.type in [aiohttp.WSMsgType.CLOSED, aiohttp.WSMsgType.ERROR]:
                    print("❌ WebSocket cerrado o con error")
                    break
    finally:
        await session.close()

# Ejecutar el bucle
await escuchar()


✅ Rayo 1 guardado
✅ Rayo 2 guardado
✅ Rayo 3 guardado
✅ Rayo 4 guardado
✅ Rayo 5 guardado
✅ Rayo 6 guardado
✅ Rayo 7 guardado
✅ Rayo 8 guardado
✅ Rayo 9 guardado
✅ Rayo 10 guardado
✅ Se han procesado 10 rayos.
